In [1]:
import pandas as pd
import numpy as np

import keras
import keras.backend as K
from keras import layers, models, optimizers
from keras.regularizers import L1L2
from keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.losses import Huber

from sklearn.preprocessing import LabelEncoder

import gc
import os



Using TensorFlow backend.


In [4]:
tr = pd.read_csv('./data/train.csv')
te = pd.read_csv('./data/test.csv')

sub = pd.read_csv('./data/sample_submission.csv', index_col=0)

In [5]:
# target_lbe = LabelEncoder().fit(target)

# t = target_lbe.transform(target)

column_number = {}
for i, column in enumerate(sub.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

tr['type_num'] = tr['type'].apply(lambda x: to_number(x, column_number))

target = tr['type_num']
t = target.copy()

train_X = tr.drop(['id', 'type', 'type_num'], axis=1)
test_X = te.drop(['id',], axis=1)


In [6]:
train_X = (train_X - np.mean(train_X))/np.std(train_X)
test_X = (test_X - np.mean(test_X))/np.std(test_X)

In [7]:
es = EarlyStopping(patience=20, restore_best_weights=True)

In [8]:
def DenseBlock(n, activation='relu'):
    def f(x):
        x = layers.Dense(n, kernel_initializer='he_normal', kernel_regularizer=L1L2(l2=0.001))(x)
#         x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        
        return x
    return f


In [9]:
K.clear_session()

inputs = layers.Input(shape=(train_X.shape[1], ))

x1 = layers.Dense(256*4, kernel_initializer='he_normal', kernel_regularizer=L1L2(l2=0.0001))(inputs)
x1 = layers.BatchNormalization()(x1)
x1 = layers.Activation('relu')(x1)

x1 = DenseBlock(256*4)(x1)
# x1 = DenseBlock(256*3)(x1)
x1 = DenseBlock(256*2)(x1)
x1 = DenseBlock(256)(x1)
x1 = DenseBlock(64)(x1)
# x1 = DenseBlock(32)(x1)

x2 = layers.Dense(256*2, kernel_initializer='he_normal', kernel_regularizer=L1L2(l2=0.0001))(inputs)
x2 = layers.BatchNormalization()(x2)
x2 = layers.Activation('relu')(x2)

x2 = DenseBlock(256, 'tanh')(x2)
# x2 = DenseBlock(256, 'tanh')(x2)
x2 = DenseBlock(128, 'tanh')(x2)
x2 = DenseBlock(64, 'tanh')(x2)
# x2 = DenseBlock(32, 'tanh')(x2)


x = layers.add([x1, x2])

# x = layers.Dense(32, activation='relu')(x)

x = layers.Dense(19, activation='softmax')(x)

m = models.Model(inputs, x)

# sparse_categorical_crossentropy
m.compile(optimizer = optimizers.Adam(lr=1e-4),
         loss = 'sparse_categorical_crossentropy', 
          metrics = ['acc'])

In [ ]:
m.fit(train_X, t, 
     epochs = 1000,
     validation_split=0.1,
     batch_size=1024*16,
      callbacks=[es]
     )

In [ ]:
m.evaluate(train_X, t)

In [ ]:
sub[sub.columns[1:]] = m.predict(test_X)

In [ ]:
sub.to_csv('./sub/nn2.csv', index=False)